In [15]:
!pip install sentence-transformers faiss-cpu

# ——— 1. Установка зависимостей ———


In [20]:
!pip install -q scikit-learn pandas



# ——— 2. Импорт библиотек ———

In [28]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ——— 3. Загрузка JSON ———

In [29]:
json_path = 'qa_data.json'
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)


# ——— 4. Нормализация и подготовка DataFrame ———

In [30]:
questions = []
answers = []
for entry in data:
    q = entry.get('question') or entry.get('Вопрос')
    a = entry.get('content') or entry.get('Ответ')
    if q and a:
        questions.append(q)
        answers.append(a)

df = pd.DataFrame({'question': questions, 'answer': answers})
df.head()  # просмотр первых строк

,question,answer
0,выйти,Для оформления обращения в техническую поддерж...
1,Уберите этот чат бот он очень мешает,Чат-бот находится в стадии пилотирования и обу...
2,вернуть старый дизайн,Чат-бот находится в стадии пилотирования и обу...
3,хочу создать заявку,Для оформления обращения в техническую поддерж...
4,мне нужно создать заявку,Для оформления обращения в техническую поддерж...


# ——— 5. Сохранение в CSV ———

In [31]:
csv_path = 'qa_data_prepared.csv'
df.to_csv(csv_path, index=False)
print(f"Сохранено в {csv_path}")

Сохранено в qa_data_prepared.csv


# ——— 6. Построение TF-IDF матрицы ———

In [32]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['question'])

# ——— 7. Функция семантического поиска ———

In [33]:
def semantic_search(query, top_n=5):
    """
    Возвращает top_n наиболее близких вопросов и их ответы.
    """
    query_vec = vectorizer.transform([query])
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_idxs = sims.argsort()[-top_n:][::-1]
    results = []
    for idx in top_idxs:
        results.append({
            'question': df.at[idx, 'question'],
            'answer': df.at[idx, 'answer'],
            'similarity': float(sims[idx])
        })
    return pd.DataFrame(results)


# ——— 8. Пример использования ———

In [34]:
query = "как добавить автомобиль"
print("Результаты поиска для:", query)
print(semantic_search(query, top_n=3))


Результаты поиска для: как добавить автомобиль
                                        question  \
0                            добавить автомобиль   
1  Не отображается автомобиль в личном кабинете.   
2                        как добавить снилс в лк   

                                              answer  similarity  
0  Для внесения данных по личному автомобилю обра...    0.952403  
1  Для внесения данных по личному автомобилю обра...    0.511704  
2  Для изменения персональных данных (адрес/образ...    0.469781  
